# MWS Secondary Target Selection

## MWS_SPECIAL_BHB_SV

Maintainers:
  - Sergey Koposov (selection/catalog)
  - Andrew Cooper (secondary target code)
  
See `/project/projectdirs/desi/target/secondary/README` for output data model.

In [1]:
!pwd
!ls ../raw

/global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_RRLYR_SV/1/code
gaiarr_source_ids_pos_pm.fits


In [3]:
# Parameters

target_class   = 'MWS_SPECIAL_BHB_SV'
input_filename = 'bhb_191127.fits'

REF_CAT        = 'G'
REF_EPOCH      = 2015.5

colmap = {
    'ra':        'RA',
    'dec':       'DEC',
    'pmra':      'PMRA',
    'pmdec':     'PMDEC',
    'source_id': 'REF_ID'
}

See:
https://github.com/segasai/mynotebooks/blob/master/desi_bhb_sel/BhbSel.ipynb

```sql
with x as ( 
 select ref_id as source_id,ref_epoch,gaia_phot_g_mean_mag,ebv,ra,dec,
       flux_g,flux_r,flux_z,pmra,pmdec,parallax,parallax_ivar,  
       -2.5*log(flux_g/flux_r)-(3.237-2.176)*ebv as gr,
       -2.5*log(flux_r/flux_z) -(2.176-1.217)*ebv as rz,
       22.5-2.5*log(flux_g) - 3.237*ebv as g,
       22.5-2.5*log(flux_r) - 2.176*ebv as r,
       22.5-2.5*log(greatest(flux_w1 
         -3/sqrt(greatest(flux_ivar_w1,1e-30)),1e-30))-  0.19*ebv as w1_faint
       from decals_dr8.main as d where 
       flux_g>0 and flux_r>0 and flux_z>0 and         
       type='PSF' and 
       brick_primary and
       gaia_phot_g_mean_mag>10 and 
        (parallax)<0.1+3*sqrt(parallax_ivar) and 
       -2.5*log(flux_g/flux_r)-(3.237-2.176)*ebv  between -0.35 and -0.02
       and fracmasked_g<0.5 and fracmasked_r<0.5 and fracmasked_z<0.5
       and not gaia_duplicated_source
       and gaia_astrometric_excess_noise<3
       and 22.5-2.5*log(flux_r) between 15 and 20
       ) 
select * from x  where  
       rz- (1.07163*gr^5-1.42272*gr^4+0.69476*gr^3
       -0.12911*gr^2+0.66993*gr-0.11368) between -0.1 and 0.05
       and r-2.3*(gr) -w1_faint < -1.5
```

In [4]:
from   importlib import reload
import sys
PYPATH = '/project/projectdirs/desi/external/mws/py/'
if PYPATH not in sys.path: sys.path.append(PYPATH)
import mwsecondary.mwsecondary as mws
reload(mws)

import os
import sys
import numpy as np

targets = mws.SecondaryTargets(target_class,input_filename,ref_cat=REF_CAT,ref_epoch=REF_EPOCH)
targets.print_info()
mask    = np.repeat(True,len(targets.data))
mask   &= targets.select_finite_range('dec',lower_limit=-30.0,verbose=True)
mask   &= targets.select_unique('source_id',verbose=True)

# Adjust to secondary target data model and write output
output = targets.get_output(mask,cols=colmap,write=True)
print(output[0:3])

Reading input: /global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_BHB_SV/1/raw/bhb_191127.fits
Read 84472 rows
First 3 rows:
     source_id             ra        ...  parallax gaia_phot_g_mean_mag
------------------- ---------------- ... --------- --------------------
4696633981847723904 36.7215534719594 ... 0.0959313               15.518
4696663286409415040 36.8859609931906 ... 0.0286209              18.2441
4696667409577705472 37.4562851342577 ... -0.453419              19.7155

Column names:
['source_id', 'ra', 'dec', 'pmra', 'pmdec', 'parallax', 'gaia_phot_g_mean_mag']

Selecting 64576 from 84472 (76.45%) on dec
83764 of 84472 targets have unique source_id
Removing 708 duplicates

Writing 63868 rows of output to /global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_BHB_SV/1/indata/MWS_SPECIAL_BHB_SV.fits
       RA              DEC          PMRA   ... OVERRIDE REF_CAT REF_EPOCH
---------------- ---------------- -------- ... -------- ------- ---------
285.859806302

In [12]:
targets.data[(targets.data['gaia_phot_g_mean_mag'] < 14.0)]

source_id,ra,dec,pmra,pmdec,parallax,gaia_phot_g_mean_mag
int64,float64,float64,float32,float32,float32,float32
5290005801,44.8884646351618,-10.7845516375608,-19.7,-54.4,0.0,12.413
